In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [4]:
molsys = systems['T4 lysozyme L99A']['t4_lysozyme_L99A.h5msm']

In [5]:
molsys_ref = msm.convert(molsys, to_form='molsysmt.MolSys')

In [6]:
molecular_system = molsys_ref
n_atoms = msm.get(molecular_system, n_atoms=True, skip_digestion=True)
atoms_water = msm.select(molecular_system, selection='group_type=="water"', skip_digestion=True)
atoms_not_water = msm._private.atom_indices.complementary_atom_indices(molecular_system, atoms_water)
heavy_atoms_not_water = msm.select(molecular_system, selection='atom_type!="H"', mask=atoms_not_water,
                               skip_digestion=True)
o_atoms_water = msm.select(molecular_system, selection='atom_type!="H"', mask=atoms_water,
                       skip_digestion=True)
h_atoms_not_water = [ii for ii in atoms_not_water if ii not in heavy_atoms_not_water] 
h_atoms_water = [ii for ii in atoms_water if ii not in o_atoms_water] 

In [19]:
msm.structure.get_distances(molecular_system, selection=1, selection_2=6)

Magnitude,[[[0.1634146907261444]]]
Units,nanometer


In [7]:
msm.get(molsys_ref, n_bonds=True)

2645

In [8]:
msm.build.remove_bonds(molsys_ref)

In [9]:
msm.get(molsys_ref, n_bonds=True)

0

In [10]:
aa, bb = msm.build.get_missing_bonds(molsys_ref)

templates and distances do not match


In [11]:
len(aa)

2699

In [16]:
aa[0]

[0, 1]

In [13]:
len(bb)

2645

In [15]:
msm.info(molsys_ref, element='atom', selection=[0,1,2,3,4,5,6,7,8,9])

index,id,name,type,group index,group id,group name,group type,component index,chain index,molecule index,molecule type,entity index,entity name
0,1,C,C,0,0,ACE,ACE,0,0,0,peptide,0,0
1,2,O,O,0,0,ACE,ACE,0,0,0,peptide,0,0
2,3,CH3,C,0,0,ACE,ACE,0,0,0,peptide,0,0
3,4,H1,H,0,0,ACE,ACE,0,0,0,peptide,0,0
4,5,H2,H,0,0,ACE,ACE,0,0,0,peptide,0,0
5,6,H3,H,0,0,ACE,ACE,0,0,0,peptide,0,0
6,7,N,N,1,1,MET,MET,6,0,6,peptide,2,2
7,8,H,H,1,1,MET,MET,6,0,6,peptide,2,2
8,9,CA,C,1,1,MET,MET,6,0,6,peptide,2,2
9,10,HA,H,1,1,MET,MET,6,0,6,peptide,2,2


In [14]:
for ii in aa:
    if ii not in bb:
        print(ii)

[1, 6]
[2584, 2606]
[2586, 2606]
[6, 9]
[37, 40]
[132, 146]
[144, 147]
[240, 239]
[240, 243]
[412, 409]
[433, 430]
[520, 517]
[523, 526]
[546, 545]
[597, 600]
[648, 643]
[658, 655]
[661, 664]
[717, 715]
[717, 716]
[725, 729]
[731, 730]
[783, 782]
[801, 800]
[829, 833]
[865, 864]
[1016, 1015]
[1148, 1147]
[1204, 1209]
[1213, 1212]
[1420, 1425]
[1427, 1426]
[1429, 1426]
[1478, 1475]
[1510, 1509]
[1683, 1682]
[1691, 1694]
[1705, 1710]
[1712, 1711]
[1786, 1783]
[1788, 1785]
[1806, 1809]
[1865, 1864]
[1939, 1943]
[1981, 1979]
[2038, 2036]
[2268, 2274]
[2275, 2273]
[2360, 2365]
[2366, 2364]
[2458, 2461]
[2545, 2544]
[2547, 2544]
[2606, 2609]
[2606, 2610]


In [ ]:
molsys = msm.convert(molsys, to_form='nglview.NGLWidget')

In [ ]:
n_molecules_ref, n_structures_ref = msm.get(molsys_ref, element='system', n_molecules=True, n_structures=True)

In [ ]:
msm.convert(molsys, to_form='molsysmt.Topology', get_missing_bonds=True)

In [ ]:
msm.get(molsys, element='system', n_molecules=True)

In [ ]:
n_molecules, n_structures = msm.get(molsys, element='system', n_molecules=True, n_structures=True)